In [21]:
################ Bon Code ###############
#### charger et lire un dataset [.zip] sur coolab ####

import zipfile
import os
import pandas as pd

import shutil

# # Chemins vers les fichiers zip quand je suis sur google Coolab

chemin_zip = "Bell_DNS Dataset.zip"


# Fonction pour extraire les fichiers zip
def extraire_zip(chemin_zip):
    with zipfile.ZipFile(chemin_zip, 'r') as zip_ref:
        zip_ref.extractall("extraction_temp")  # Extraire les fichiers zip dans un répertoire temporaire

# Fonction pour charger les fichiers CSV d'un type spécifique (stateful ou stateless)
def charger_concatener_donnees(sous_dossier, prefixe):
    # Lister tous les fichiers CSV dans le sous-dossier
    fichiers_csv = [f for f in os.listdir(f"extraction_temp/{sous_dossier}") if f.startswith(prefixe) and f.endswith('.csv')]
    # Lire chaque fichier CSV et le stocker dans une liste de DataFrames
    dataframes = [pd.read_csv(f"extraction_temp/{sous_dossier}/{f}") for f in fichiers_csv]
    # Concaténer les DataFrames en un seul
    return pd.concat(dataframes, ignore_index=True)

# Extraire les fichiers zip
extraire_zip(chemin_zip)



benin_data = charger_concatener_donnees("Bell_DNS Dataset", "features_domain_benign")
malware_data = charger_concatener_donnees("Bell_DNS Dataset", "features-domain_Malware")

phishing_data = charger_concatener_donnees("Bell_DNS Dataset", "features-domain_phishing")
spam_data = charger_concatener_donnees("Bell_DNS Dataset", "features-domain_Spam")

# Supprimer le répertoire temporaire après avoir terminé

# Vérifier si le répertoire temporaire existe
if os.path.exists("extraction_temp"):
    # Supprimer le répertoire temporaire et son contenu
    shutil.rmtree("extraction_temp")


# Maintenant, vous avez vos données prêtes à être utilisées

In [14]:
#### Je ne resample plus

import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

# Ajout de la colonne 'Class' pour chaque classe
spam_data['class'] = 'spam'
malware_data['class'] = 'malware'
phishing_data['class'] = 'phishing'
benin_data['class'] = 'Benin'


# Concaténation des données
all_data = pd.concat([benin_data, spam_data, malware_data, phishing_data])

# Diviser les données en caractéristiques (X) et les étiquettes de classe (y)
X = all_data.drop(columns=['class'])
y = all_data['class']

print("Nbre de classe:", all_data['class'].value_counts())


Nbre de classe: class
Benin       24249
phishing    12702
malware      8871
spam         4337
Name: count, dtype: int64


In [28]:
import pandas as pd
from sklearn.utils import shuffle

# Ajout de la colonne 'Class' pour chaque classe
spam_data['class'] = 'spam'
malware_data['class'] = 'malware'
phishing_data['class'] = 'phishing'
benin_data['class'] = 'Benin'

# Concaténation des données
all_data = pd.concat([benin_data, spam_data, malware_data, phishing_data])

# Définir la taille désirée pour la classe 'malicieux'
target_malicious_size = 22929

# Calculer le nombre d'échantillons à sélectionner pour chaque classe
target_phishing_size = int(target_malicious_size * (12702 / (12702 + 8871 + 4337)))
target_malware_size = int(target_malicious_size * (8871 / (12702 + 8871 + 4337)))
target_spam_size = int(target_malicious_size * (4337 / (12702 + 8871 + 4337)))

# Sélectionner des échantillons de chaque classe
malicious_data = all_data[all_data['class'] == 'phishing'].sample(n=target_phishing_size, random_state=42, replace=False)
malicious_data = pd.concat([malicious_data, all_data[all_data['class'] == 'malware'].sample(n=target_malware_size, random_state=42, replace=False)])
malicious_data = pd.concat([malicious_data, all_data[all_data['class'] == 'spam'].sample(n=target_spam_size, random_state=42, replace=False)])

# Si le nombre total d'échantillons est inférieur à la taille désirée, ajoutez des échantillons supplémentaires en remplaçant
while len(malicious_data) < target_malicious_size:
    missing_samples = target_malicious_size - len(malicious_data)
    additional_samples = all_data[all_data['class'] == 'phishing'].sample(n=missing_samples, random_state=42, replace=True)
    malicious_data = pd.concat([malicious_data, additional_samples])

# Si le nombre total d'échantillons est supérieur à la taille désirée, supprimez les échantillons excédentaires
malicious_data = malicious_data.sample(n=target_malicious_size, random_state=42)

# Mélanger les données
malicious_data = shuffle(malicious_data, random_state=42)

# Attribuer la classe 'malicieux'
malicious_data['class'] = 'malicieux'

print("Nombre de classes dans le nouveau jeu de données :")
print(malicious_data['class'].value_counts())



# Sélectionner des échantillons de la classe 'Benin'
target_benin_size = 23716
benin_samples = benin_data.sample(n=target_benin_size, random_state=42)

# Concaténer les échantillons de la classe 'Benin' avec le jeu de données malicieux
final_data = pd.concat([malicious_data, benin_samples])

# Mélanger les données
final_data = shuffle(final_data, random_state=42)

print("Nombre de classes dans le nouveau jeu de données :")
print(final_data['class'].value_counts())

Nombre de classes dans le nouveau jeu de données :
class
malicieux    22929
Name: count, dtype: int64
Nombre de classes dans le nouveau jeu de données :
class
Benin        23716
malicieux    22929
Name: count, dtype: int64


In [29]:
# Afficher le nombre d'échantillons pris dans chaque classe pour former le jeu de données malicieux
print("Nombre d'échantillons pris dans chaque classe pour former le jeu de données malicieux :")
print(f"Phishing : {target_phishing_size}")
print(f"Malware : {target_malware_size}")
print(f"Spam : {target_spam_size}")


Nombre d'échantillons pris dans chaque classe pour former le jeu de données malicieux :
Phishing : 11240
Malware : 7850
Spam : 3838
